# 식신 - 제주시청

In [1]:
# 식신 - 제주시청

In [2]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

### 1. 데이터 가져오기

In [3]:
site = 'https://www.siksinhot.com'
url = f'{site}/search?keywords={quote("제주시청")}'
req = requests.get(url)
html = req.text
type(html)

str

In [4]:
soup = BeautifulSoup(html,'html.parser')

In [5]:
# type(soup)

## # 2. 찾으려는 하는 데이터의 태그 찾기

In [6]:
# lis = soup.select_one('div.listTy1').find_all('li') 
# len(lis)

In [7]:
lis = soup.select('div.listTy1> ul > li') # 자식 Selector
len(lis)

0

### 3. 하나를 선택해서 원하는 정보 추출
- 상호, 평점, 메뉴 , 서브 페이지의 url
- 전화번호, 주소

In [8]:
li = lis[1]
href = li.find('a')['href']


IndexError: list index out of range

In [ ]:
score = li.select_one(".score").string
score

'3.4'

In [ ]:
store = li.select_one(".store").string
store

'안양돈까스 제주시점'

In [ ]:
menu = li.find('p').get_text()
menu

'돈까스 세트, 치즈돈까스 세트'

In [ ]:
score,store,menu,href

('3.4', '안양돈까스 제주시점', '돈까스 세트, 치즈돈까스 세트', '/P/466268')

In [ ]:
sub_url = site + href
sub_url


'https://www.siksinhot.com/P/466268'

In [ ]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [ ]:
tel = sub_soup.select_one('.p_tel').find('p').get_text()
tel

'064-799-1434'

In [ ]:
addr = sub_soup.select_one('.txt_adr').get_text()
addr

'제주특별자치도 제주시 애월읍 애월로 40'

In [ ]:
# try - except 를 이용해서 어디에서 에러가 발생하는지 확인하고 수정하기 
lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one(".score").string
        store = li.select_one(".store").get_text()
        menu = li.find('p').get_text()
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    except:
        print(index)
    
    lines.append([store,score,menu,tel,addr])


0


### 5. 에러난 부분을 처리할 수 있도록 수정

In [ ]:
li = lis[0]
score = li.select_one('.cnt>em').get_text()
type(score)

str

In [ ]:
li = lis[1]
score = li.select_one('.cnt>em').get_text()
type(score)

str

### 6. 에러가 해결될 때 까지 5, 6을 반복

In [9]:
lines=[]
for index, li in enumerate(lis):
    try:
        href = li.find('a')['href']
        score = li.select_one('.cnt>em').string
        if score == '평가중':
            score = None
        store = li.select_one(".store").string
        menu = li.find('p').get_text()
        
        sub_url = site + href
        sub_req = requests.get(sub_url)
        sub_html = sub_req.text
        sub_soup = BeautifulSoup(sub_html, 'html.parser')
        tel = sub_soup.select_one('.p_tel').find('p').get_text()
        addr = sub_soup.select_one('.txt_adr').get_text()
    
    except:
        print(index)
    
    lines.append([store,score,menu,tel,addr])

In [10]:
lines

[]

### 7. 데이터 프레임으로 만들기

In [11]:
df = pd.DataFrame(lines, columns=['업소명','평점','메뉴','전화번호','주소'])
df

,업소명,평점,메뉴,전화번호,주소


In [ ]:
import seaborn as s


